In [199]:
!pip install -r requirements.txt --user

In [200]:
import time

import boto3

sm = boto3.Session().client("sagemaker")

In [201]:
def get_regions():
    ec2 = boto3.client("ec2")
    regions = ec2.describe_regions()
    return [r["RegionName"] for r in regions["Regions"]]

In [202]:
def search_projects(_project):
    projects = sm.list_projects(NameContains=_project)

    if "NextToken" in projects:
        next_token = projects["NextToken"]
        print(f"\n{next_token}")

    print("ProjectNames:")
    for project in projects["ProjectSummaryList"]:
        project_name = project["ProjectName"]
        print(f"\n{project_name}")

In [203]:
def cleanup_experiment(_experiment_name):
    trials = sm.list_trials(ExperimentName=_experiment_name)["TrialSummaries"]
    print("TrialNames:")
    for trial in trials:
        trial_name = trial["TrialName"]
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print("\tTrialComponentNames:")
        for component in components_in_trial["TrialComponentSummaries"]:
            component_name = component["TrialComponentName"]
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(0.5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=_experiment_name)
    print(f"\nExperiment {_experiment_name} deleted")

In [204]:
def cleanup_model(_model_name):
    models = sm.list_models(NameContains=_model_name)
    model_bias_job_definitions = sm.list_model_bias_job_definitions(NameContains=_model_name)
    model_explainability_job_definitions = sm.list_model_explainability_job_definitions(
        NameContains=_model_name
    )
    # model_metadata = sm.list_model_metadata(NameContains=_model_name)
    model_package_groups = sm.list_model_package_groups(NameContains=_model_name)
    model_quality_job_definitions = sm.list_model_quality_job_definitions(NameContains=_model_name)

    if "NextToken" in models:
        next_token = models["NextToken"]
        print(f"\n{next_token}")

    print("JobDefinitions:")
    for model in model_bias_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    print("JobDefinitions:")
    for model in model_explainability_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    # print("ModelNames:")
    # for model in model_metadata["Models"]:
    #     model_name = model["ModelName"]
    #     print(f"\n{model_name}")

    print("ModelPackageGroupNames:")
    for group in model_package_groups["ModelPackageGroupSummaryList"]:
        group_name = group["ModelPackageGroupName"]
        print(f"\n{group_name}")

        model_packages = sm.list_model_packages(ModelPackageGroupName=group_name)
        print("ModelPackageNames:")
        for package in model_packages["ModelPackageSummaryList"]:
            if "ModelPackageName" in package:
                package_name = package["ModelPackageName"]
                print(f"\n{package_name}")
                sm.delete_model_package(ModelPackageName=package_name)
        sm.delete_model_package_group(ModelPackageGroupName=group_name)

    print("JobDefinitions:")
    for model in model_quality_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    print("ModelNames:")
    for model in models["Models"]:
        model_name = model["ModelName"]
        print(f"\n{model_name}")

In [219]:
def cleanup_endpoint(_endpoint_name):
    endpoints = sm.list_endpoints(NameContains=_endpoint_name)

    if "NextToken" in endpoints:
        next_token = endpoints["NextToken"]
        print(f"\n{next_token}")

    print("EndpointNames:")
    for endpoint in endpoints["Endpoints"]:
        endpoint_name = endpoint["EndpointName"]
        sm.delete_endpoint(EndpointName=endpoint_name)
        print(f"\n{endpoint_name} deleted")

    #     components_in_trial = sm.list_trial_components(TrialName=trial_name)
    #     print("\tTrialComponentNames:")
    #     for component in components_in_trial["TrialComponentSummaries"]:
    #         component_name = component["TrialComponentName"]
    #         print(f"\t{component_name}")
    #         sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
    #         try:
    #             # comment out to keep trial components
    #             sm.delete_trial_component(TrialComponentName=component_name)
    #         except:
    #             # component is associated with another trial
    #             continue
    #         # to prevent throttling
    #         time.sleep(0.5)
    #     sm.delete_trial(TrialName=trial_name)
    # sm.delete_experiment(ExperimentName=_experiment_name)
    # print(f"\nExperiment {_experiment_name} deleted")

In [220]:
def cleanup_monitoring_schedules(_monitoring_schedule_name):
    sm.delete_monitoring_schedule(MonitoringScheduleName=_monitoring_schedule_name)

In [206]:
def cleanup_endpoints(_endpoint_name):
    sm.delete_endpoint(EndpointName=_endpoint_name)

In [207]:
def cleanup_project(project_name):
    sm.delete_project(ProjectName=project_name)

In [208]:
experiment_name = "Experiment-1"
cleanup_experiment(experiment_name)

monitoring_schedules = sm.list_monitoring_schedules()
for monitoring_schedule in monitoring_schedules["MonitoringScheduleSummaries"]:
    print(monitoring_schedule["MonitoringScheduleName"])
    monitoring_schedule_name = monitoring_schedule["MonitoringScheduleName"]
    cleanup_monitoring_schedules(monitoring_schedule_name)

endpoints = sm.list_endpoints()
for endpoint in endpoints["Endpoints"]:
    print(endpoint["EndpointName"])
    endpoint_name = endpoint["EndpointName"]
    cleanup_endpoints(endpoint_name)

ResourceNotFound: An error occurred (ResourceNotFound) when calling the ListTrials operation: Experiment 'arn:aws:sagemaker:ap-southeast-2:487750889657:experiment/experiment-1' does not exist.

In [209]:
# get_regions()

In [210]:
search_projects("animal")

ProjectNames:

animal-classifier


In [211]:
cleanup_model("AbaloneModel")


cIws2QhTXUIa8bi8WfGtdPps9v88eG63iCQLqSXVJZs49aU70ECEWZ+dm+ceK1BBww0cOrp/9KWwuxqPb6YVZWIRXKuIMZYd8TbcDiv4snGlfCc6d5Yh13klD7aGT9V/eeehnWAIdWc+qamwcaiORvsTEJC62kkD4oUw9owZAX5RPCAsxxa5/nwDBQUt+tpupsiDvPuGFGtQF7ud9PGRuc9xWbXTJLGo/39Ce2w4jAgcRaDwK84H85kLs2Xg8gt+PfNZHnhbBrvcp36AU+U8+JQssVPSe6zTBa48Vp7Q2TSr6l/7D+mz6TudsDxOsrsmIkMxPmFvb9oS38Lsw36grAZR05dPlwb+VhzXBgVJfn5YsRYM9p49zDuiUgWsPjGf6J5N6zH3kWbVVEpGSCBGyWnQOTGJ6PG+9rjAn/frFLCz2fhQ/qsn0TZOiVk6RTSk8qoXemZEFjngIbuFWv/tI5XavBaCEXyDGo80civk4EaaBZ+ZKOzWJZo7dgQ5O6gld7P08VoMeq1diNLg3Wn1kp401hFgn5tkQjvItP3z0pMUNylrpWEowqf89W399vJs8K6y4GuTn5kJQCM0hG6+CkYV6ow5CExEFkcXfH0NMlPcmo3Xeg==
JobDefinitions:
JobDefinitions:
ModelPackageGroupNames:
JobDefinitions:
ModelNames:


In [212]:
cleanup_model("animal-classifier")


cIws2QhTXUIa8bi8WfGtdPps9v88eG63iCQLqSXVJZs49aU70ECEWZ+dm+ceK1BBww0cOrp/9KWwuxqPb6YVZWIRXKuIMZYd8TbcDiv4snGlfCc6d5Yh13klD7aGT9V/eeehnWAIdWc+qamwcaiORvsTEJC62kkD4oUw9owZAX5RPCAsxxa5/nwDBQUt+tpupsiDvPuGFGtQF7ud9PGRuc9xWbXTJLGo/39Ce2w4jAgcRaDwK84H85kLs2Xg8is0oaqwdsugM0syDnLQmCYaJAaP91qTij+SUc3kj/kG2ctKZPH5/lFNKnpbYfazTkmBFZ7KW6/Ke2OfN1w8mcjaPBdbjOOVSwNYlVTPLpSTG85wtEfDe0pAMVrNvhL7xkbv0rsvrcn+rYCrM34foQsN035zVyUYgnEu40DYHoJ6AD3HVUAQNmPN6/1g4QDUepPu4DI8sWAmB7UUOzwa1Y4Rj8CsSjPGg9cDhJn1eLMcIcyrNwRLeO7MM3VofW5bRtfl/wSchsxtfve5CtI20DNRlPGLoe8yxf8tMw/QVEjTYVIQjzjXiNqKmox+9RF1Cbz4a7UdSGIoWPetKZwlTkSR74RNS1UjEYz5celzevPqo5NN1yvVw+Bh2bbi
JobDefinitions:
JobDefinitions:
ModelPackageGroupNames:
JobDefinitions:
ModelNames:


In [213]:
cleanup_project("animal-classifier")

In [221]:
cleanup_endpoint("sagemaker-sklearn-automl")

EndpointNames:

sagemaker-sklearn-automl-2022-02-21-07-08-03-300 deleted

sagemaker-sklearn-automl-2022-02-21-04-23-13-518 deleted

sagemaker-sklearn-automl-2022-02-17-07-43-41-060 deleted
